In [53]:
%%time
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
import re
import nltk

CPU times: total: 0 ns
Wall time: 997 µs


In [54]:
raw_train_data = pd.read_excel('train.xlsx', index_col = 0) 
raw_val_data = pd.read_excel('val.xlsx', index_col = 0)
raw_test_data = pd.read_excel('test.xlsx', index_col = 0)

In [56]:
raw_train_data.head()

,text,classes,labels
0,মেডিকেলে এসে ঠিক ফ্যামিলির ফটো ফ্রেম এর মতো মন...,sadness,1
1,অনেক দিন পর অন্য রকম অনুভূতি হচ্ছে কেননা আমার...,sadness,1
2,আগে যতোটা বেশি পছন্দ করতাম দুশ্চরিত্রা মিথিলা ...,disgust,3
3,মাঝে মাঝে মনে হয় তোমাকে চাওয়াটা অনেক ভুল হয়ত ভ...,sadness,1
4,এখানে কাওকে কবর দিয়ো না তাহলে তোমাদের পরিবারের...,fear,5


In [57]:
raw_val_data['text'][0]

'তোরা সবাই বিটিভির পর্দায় ফিরে আয়🙂তোদের জন্য মুভি নারেবাম্পার ফলনই তোদের বিনোদনের মূল মাধ্যম 🙄পোলাপান কতটা সিরিয়াস মাইরিভাবা যায় সামান্য মুভির সামান্য রিভিউ বিনিময়ে কী সাংঘাতিক গালাগাল🙄ম্যাঁ গো'

In [58]:
#The Bengali Characters

from bltk.langtools.banglachars import (vowels,
                                        vowel_signs,
                                        consonants,
                                        digits,
                                        operators,
                                        punctuations,
                                        others)
print(f'Vowels: {vowels}')
print(f'Vowel signs: {vowel_signs}')
print(f'Consonants: {consonants}')
print(f'Digits: {digits}')
print(f'Operators: {operators}')
print(f'Punctuation marks: {punctuations}')
print(f'Others: {others}')

Vowels: ['অ', 'আ', 'ই', 'ঈ', 'উ', 'ঊ', 'ঋ', 'ঌ', 'এ', 'ঐ', 'ও', 'ঔ']
Vowel signs: ['া', 'ি', 'ী', 'ু', 'ূ', 'ৃ', 'ৄ', 'ে', 'ৈ', 'ো', 'ৌ']
Consonants: ['ক', 'খ', 'গ', 'ঘ', 'ঙ', 'চ', 'ছ', 'জ', 'ঝ', 'ঞ', 'ট', 'ঠ', 'ড', 'ঢ', 'ণ', 'ত', 'থ', 'দ', 'ধ', 'ন', 'প', 'ফ', 'ব', 'ভ', 'ম', 'য', 'র', 'ল', 'শ', 'ষ', 'স', 'হ', 'ড়', 'ঢ়', 'য়', 'ৎ', 'ং', 'ঃ', 'ঁ']
Digits: ['০', '১', '২', '৩', '৪', '৫', '৬', '৭', '৮', '৯']
Operators: ['=', '+', '-', '*', '/', '%', '<', '>', '×', '÷']
Punctuation marks: ['।', ',', ';', ':', '?', '!', "'", '.', '"', '-', '[', ']', '{', '}', '(', ')', '–', '—', '―', '~']
Others: ['৳', '৺', '্', 'ঀ', 'ঽ', '#', '$']


In [59]:
from bltk.langtools import Tokenizer, remove_stopwords
from bangla_stemmer.stemmer import stemmer

In [60]:
def preprocess_bangla_text(text):
    
    #Tokenize the text
    tokenizer = Tokenizer()
    words = tokenizer.word_tokenizer(text)
    
    #Remove Stopword
    filtered_words = remove_stopwords(words)
    
    #Stem Words:
    stmer = stemmer.BanglaStemmer()
    stemmed_words = [stmer.stem(word) for word in filtered_words]
    stemmed_text = " ".join(stemmed_words)
    
    #remove HTML tag from the text
    html_tag = re.compile(r'<.*?>')
    html_clean_text = re.sub(html_tag,'',stemmed_text)
    
    #remove url
    url_clean = re.compile(r"https://\S+|www\.\S+")
    url_clean_text = re.sub(url_clean,'',html_clean_text)
    
    #remove emojies
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F700-\U0001F77F"  # alchemical symbols
                           u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                           u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                           u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                           u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                           u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                           u"\U0001FAE0-\U0001FAFF"  # Symbols for Legacy Computing
                           u"\U0001FAC0-\U0001FAFF"  # Additional emoticons
                           u"\U0001F004-\U0001F0CF"  # Miscellaneous Symbols and Pictographs
                           u"\U0001F170-\U0001F251"  # Enclosed Alphanumeric Supplement
                           "]+", flags=re.UNICODE)
    emoji_clean_text = re.sub(emoji_pattern,'',url_clean_text)
    
    preprocessed_text = emoji_clean_text
    return preprocessed_text
    
    
    
    
    
    
    

In [61]:
text = "তোরা সবাই বিটিভির পর্দায় ফিরে আয়🙂তোদের জন্য মুভি নারেবাম্পার ফলনই তোদের বিনোদনের মূল মাধ্যম 🙄পোলাপান কতটা সিরিয়াস মাইরিভাবা যায় সামান্য মুভির সামান্য রিভিউ বিনিময়ে কী সাংঘাতিক গালাগাল🙄ম্যাঁ গো"

In [62]:
print(preprocess_bangla_text(text))

applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
তোরা সবা বিটিভির পর্দায় ফিরে আয়তো মুভি নারেবাম্পার ফলন তোদের বিনোদন মূল মাধ্যম পোলাপান কত সিরিয়াস মাইরিভাবা সামান্য মুভির সামান্য রিভিউ বিনিময় সাংঘাতিক গালাগালম্যাঁ গো


In [63]:
raw_train_data['clean_text'] = raw_train_data['text'].apply(preprocess_bangla_text)

applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied fo

applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied first 

applied fourth rules..
applied second rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied second rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
app

applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied second rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied 

applied third rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied third rules..
applied second rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied second rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
appli

...

applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied second rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied first rules..
applied second rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied

applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied third rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied fi

applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied second rules..
applied second rules..
applied second rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied second rules..
applied second rules..
applied first rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied third rules..
applied third rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied

applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied second rules..
applied first rules..
applied first rules..
applied third rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..


applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied first rules..
applied second rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied s

In [64]:
raw_train_data.head()

,text,classes,labels,clean_text
0,মেডিকেলে এসে ঠিক ফ্যামিলির ফটো ফ্রেম এর মতো মন...,sadness,1,মেডিকেল এসে ফ্যামিলির ফটো ফ্রেম মত মনে হত নিজে...
1,অনেক দিন পর অন্য রকম অনুভূতি হচ্ছে কেননা আমার...,sadness,1,অনেক দিন পর অন্য রকম অনুভূতি কেননা সদ্য পাশ কর...
2,আগে যতোটা বেশি পছন্দ করতাম দুশ্চরিত্রা মিথিলা ...,disgust,3,যতো বেশি পছন্দ করতাম দুশ্চরিত্ মিথিলা মুখার্জি...
3,মাঝে মাঝে মনে হয় তোমাকে চাওয়াটা অনেক ভুল হয়ত ভ...,sadness,1,মনে তোমাকে চাওয়া অনেক ভুল হয়ত ভুল তবে ভুল মাশু...
4,এখানে কাওকে কবর দিয়ো না তাহলে তোমাদের পরিবারের...,fear,5,কাওকে কবর দিয়ো তোমা পরিবার আর কেউ বেঁচে থাকবে ...


## extract feature:

In [13]:
train_labels = raw_train_data.labels.tolist()

In [14]:
for i in range(5):
    print(train_texts[i])

মেডিকেল এসে ফ্যামিলির ফটো ফ্রেম মত মনে হত নিজেকেখাপছাড়া লাগতোদম হয় আসতোঅন্য সামনে অপমানিত হত পদে পদে
অনেক দিন পর অন্য রকম অনুভূতি কেননা সদ্য পাশ কর শিক্ষার্থীর মত এখন আমি বেক বেক পরিচয় দিয়ে আম্ম কাছ আজ পঞ্চাশ টাকা ডাকাতি কর
যতো বেশি পছন্দ করতাম দুশ্চরিত্ মিথিলা মুখার্জি এখন তেমনি সবচ বেশি ঘৃণা করি ওই পতিতা সদ্য হিন্দু হওয়া মিথিলা মুখার্জি
মনে তোমাকে চাওয়া অনেক ভুল হয়ত ভুল তবে ভুল মাশুল এইভাবেই দিত হত জানি তবে যা জানি মানি
কাওকে কবর দিয়ো তোমা পরিবার আর কেউ বেঁচে থাকবে ও তোমা কাউ বাঁচ দিবে চলে যাও এখান নিজে অভিশাপ আহ্বান করো


In [15]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

CPU times: total: 234 ms
Wall time: 2.51 s


# Embedding and model learning with tf-idf vectorizer

In [ ]:
lr_model = LogisticRegressionCV(class_weight='balanced', max_iter=400)
dt_model = DecisionTreeClassifier(class_weight='balanced')
rf_model = RandomForestClassifier(class_weight='balanced')
svm_model = SVC(class_weight='balanced')
knn_model = KNeighborsClassifier()
model_names = ['Logistic Regression','Decision Tree','Random Forest', 'SVM', 'KNN']
ml_models = [lr_model,dt_model,rf_model, svm_model, knn_model]

In [ ]:
%%time
def model_train(model, train_data, train_labels):
  pipe = Pipeline([
        ('FE', TfidfVectorizer(ngram_range=(1,3), use_idf = True, tokenizer=lambda x: x.split(), max_features=20000)),  
        ('Transformer', TfidfTransformer()),  
        ('M', model)
    ])
  pipe.fit(train_data, train_labels)
  return pipe

trained_ml_models = []
for i in ml_models:
    tt = model_train(i,train_texts, train_labels)
    trained_ml_models.append(tt)

# Test the models for Test and Validation data set

In [65]:
raw_test_data['clean_text'] = raw_test_data['text'].apply(preprocess_bangla_text)
raw_val_data['clean_text'] = raw_val_data['text'].apply(preprocess_bangla_text)

applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied second rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
app

applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applie

applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied third rules..
applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied third rules..
applied first rules..
applied fourth rules..
applied first rules..
applied third rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied second rules..
applied second rules..
applied fourth rules..
applied second rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
app

applied first rules..
applied first rules..
applied second rules..
applied first rules..
applied first rules..
applied second rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fou

applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied second rules..
applied first rules..
applied second rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
appl

...

applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied 

applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied second rules..
applied first rules..
applied 

applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied third rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied second rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied

applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied third rules..
applied first rules..
applied fourth rules..
applied first rules..
applied four

applied fourth rules..
applied second rules..
applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied second rules..


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
from sklearn.metrics import average_precision_score,roc_auc_score, roc_curve, precision_recall_curve
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

## Test Dataset

In [66]:
test_texts = raw_test_data.clean_text.tolist()
test_labels = raw_test_data.labels.tolist()

In [ ]:
##test data
pr_list = []
re_list = []
f1_list = []
accuracy_list = []


def print_results(md, x_test, y_test, name):
#     print("\n\nClassifier: ", name)
    pred_y = md.predict(x_test)
    true, pred = y_test, pred_y
#     print(confusion_matrix(true,pred))
#     print(classification_report(true,pred))
#     print("\nPrecison : ",precision_score(pred,true, average = 'weighted'))
#     print("\nRecall : ",recall_score(pred,true,  average = 'weighted'))
#     print("\nF1 : ",f1_score(pred,true,  average = 'weighted'))

    pr_list.append((round(precision_score(y_test, pred_y, average='weighted'),4)*100)) 
    re_list.append((round(recall_score(y_test, pred_y, average='weighted'),4)*100))
    f1_list.append((round(f1_score(y_test, pred_y, average='weighted'),4)*100))
    accuracy_list.append(accuracy_score(y_test, pred_y, normalize=True)*100)

for i in range(len(trained_ml_models)):
    md = trained_ml_models[i]
    name = model_names[i]
    print_results(md, test_texts, test_labels, name)

performance_matrix = pd.DataFrame({'Precision':pr_list,
                                   'Recall':re_list,'F1 Score':f1_list, 'Accuracy':accuracy_list},
                                  index =model_names)
performance_matrix

## Validation Dataset

In [73]:
val_texts = raw_val_data.clean_text.tolist()
val_labels = raw_val_data.labels.tolist()

In [ ]:
##val data
pr_list = []
re_list = []
f1_list = []
accuracy_list = []


def print_results(md, x_test, y_test, name):
#     print("\n\nClassifier: ", name)
    pred_y = md.predict(x_test)
    true, pred = y_test, pred_y
#     print(confusion_matrix(true,pred))
#     print(classification_report(true,pred))
#     print("\nPrecison : ",precision_score(pred,true, average = 'weighted'))
#     print("\nRecall : ",recall_score(pred,true,  average = 'weighted'))
#     print("\nF1 : ",f1_score(pred,true,  average = 'weighted'))

    pr_list.append((round(precision_score(y_test, pred_y, average='weighted'),4)*100)) 
    re_list.append((round(recall_score(y_test, pred_y, average='weighted'),4)*100))
    f1_list.append((round(f1_score(y_test, pred_y, average='weighted'),4)*100))
    accuracy_list.append(accuracy_score(y_test, pred_y, normalize=True)*100)

for i in range(len(trained_ml_models)):
    md = trained_ml_models[i]
    name = model_names[i]
    print_results(md, val_texts,val_labels, name)

performance_matrix = pd.DataFrame({'Precision':pr_list,
                                   'Recall':re_list,'F1 Score':f1_list, 'Accuracy':accuracy_list},
                                  index =model_names)
performance_matrix

# ML model with with FastText Embadding

In [ ]:
%%time
import fasttext.util
import fasttext
import numpy as np

# Load pre-trained FastText embeddings for Bengali
model_fasttext = fasttext.load_model('cc.bn.300.bin')  # Load the FastText model

In [ ]:
%%time
fasttext_text_embeddings = []

# Iterate through each text
for text in train_texts:
    tokens = text.split()  # Tokenize the text (split by space, you can use a more advanced tokenizer)
    token_embeddings = []  # To store embeddings for individual tokens in the text
    for token in tokens:
        token_embedding = model_fasttext.get_word_vector(token)  # Generate FastText embedding for the token
        token_embeddings.append(token_embedding)
    # Calculate the mean of token embeddings to represent the entire text
    if token_embeddings:
        text_embedding = np.mean(token_embeddings, axis=0)
        fasttext_text_embeddings.append(text_embedding)
    else:
        # Handle cases where there are no tokens (empty text)
        fasttext_text_embeddings.append(np.zeros(300))

In [ ]:
%%time
def model_train(model, train_data, train_labels):
    pipe_fasttext = Pipeline([
        ('M', model)
        ])
    pipe_fasttext.fit(train_data, train_labels)
    return pipe_fasttext

trained_ml_models = []
for i in ml_models:
    tt = model_train(i,fasttext_text_embeddings, train_labels)
    trained_ml_models.append(tt)

# Test Dataset

In [ ]:
test_texts = raw_test_data.clean_text.tolist()
test_labels = raw_test_data.labels.tolist()

In [ ]:
fasttext_text_embeddings = []

# Iterate through each text
for text in test_texts:
    tokens = text.split()  # Tokenize the text (split by space, you can use a more advanced tokenizer)
    token_embeddings = []  # To store embeddings for individual tokens in the text
    for token in tokens:
        token_embedding = model_fasttext.get_word_vector(token)  # Generate FastText embedding for the token
        token_embeddings.append(token_embedding)
    # Calculate the mean of token embeddings to represent the entire text
    if token_embeddings:
        text_embedding = np.mean(token_embeddings, axis=0)
        fasttext_text_embeddings.append(text_embedding)
    else:
        # Handle cases where there are no tokens (empty text)
        fasttext_text_embeddings.append(np.zeros(300))

In [ ]:
##test data
pr_list = []
re_list = []
f1_list = []
accuracy_list = []


def print_results(md, x_test, y_test, name):
#     print("\n\nClassifier: ", name)
    pred_y = md.predict(x_test)
    true, pred = y_test, pred_y
#     print(confusion_matrix(true,pred))
#     print(classification_report(true,pred))
#     print("\nPrecison : ",precision_score(pred,true, average = 'weighted'))
#     print("\nRecall : ",recall_score(pred,true,  average = 'weighted'))
#     print("\nF1 : ",f1_score(pred,true,  average = 'weighted'))

    pr_list.append((round(precision_score(y_test, pred_y, average='weighted'),4)*100)) 
    re_list.append((round(recall_score(y_test, pred_y, average='weighted'),4)*100))
    f1_list.append((round(f1_score(y_test, pred_y, average='weighted'),4)*100))
    accuracy_list.append(accuracy_score(y_test, pred_y, normalize=True)*100)

for i in range(len(trained_ml_models)):
    md = trained_ml_models[i]
    name = model_names[i]
    print_results(md, fasttext_text_embeddings, test_labels, name)

performance_matrix = pd.DataFrame({'Precision':pr_list,
                                   'Recall':re_list,'F1 Score':f1_list, 'Accuracy':accuracy_list},
                                  index =model_names)
performance_matrix

# validation dataset

In [ ]:
val_texts = raw_val_data.clean_text.tolist()
val_labels = raw_val_data.labels.tolist()

In [ ]:
fasttext_text_embeddings = []

# Iterate through each text
for text in val_texts:
    tokens = text.split()  # Tokenize the text (split by space, you can use a more advanced tokenizer)
    token_embeddings = []  # To store embeddings for individual tokens in the text
    for token in tokens:
        token_embedding = model_fasttext.get_word_vector(token)  # Generate FastText embedding for the token
        token_embeddings.append(token_embedding)
    # Calculate the mean of token embeddings to represent the entire text
    if token_embeddings:
        text_embedding = np.mean(token_embeddings, axis=0)
        fasttext_text_embeddings.append(text_embedding)
    else:
        # Handle cases where there are no tokens (empty text)
        fasttext_text_embeddings.append(np.zeros(300))

In [ ]:
##val data
pr_list = []
re_list = []
f1_list = []
accuracy_list = []


def print_results(md, x_test, y_test, name):
#     print("\n\nClassifier: ", name)
    pred_y = md.predict(x_test)
    true, pred = y_test, pred_y
#     print(confusion_matrix(true,pred))
#     print(classification_report(true,pred))
#     print("\nPrecison : ",precision_score(pred,true, average = 'weighted'))
#     print("\nRecall : ",recall_score(pred,true,  average = 'weighted'))
#     print("\nF1 : ",f1_score(pred,true,  average = 'weighted'))

    pr_list.append((round(precision_score(y_test, pred_y, average='weighted'),4)*100)) 
    re_list.append((round(recall_score(y_test, pred_y, average='weighted'),4)*100))
    f1_list.append((round(f1_score(y_test, pred_y, average='weighted'),4)*100))
    accuracy_list.append(accuracy_score(y_test, pred_y, normalize=True)*100)

for i in range(len(trained_ml_models)):
    md = trained_ml_models[i]
    name = model_names[i]
    print_results(md, fasttext_text_embeddings,val_labels, name)

performance_matrix = pd.DataFrame({'Precision':pr_list,
                                   'Recall':re_list,'F1 Score':f1_list, 'Accuracy':accuracy_list},
                                  index =model_names)
performance_matrix

# ML model with text embadding with GLOBE dataset

In [ ]:
glove_model = {}

with open('bn_glove.39M.300d.txt', 'r', encoding='utf-8') as file:
    for line in file:
        parts = line.strip().split()
        word = parts[0]
        embedding = np.array(parts[1:], dtype=np.float32)
        glove_model[word] = embedding

In [ ]:
%%time
globe_text_embeddings = []

for text in train_texts:
    tokens = text.split()
    token_embeddings = []
    for token in tokens:
        if token in glove_model:
            token_embedding = glove_model[token]
            token_embeddings.append(token_embedding)
    if token_embeddings:
        text_embedding = np.mean(token_embeddings, axis=0)
        globe_text_embeddings.append(text_embedding)
    else:
        globe_text_embeddings.append(np.zeros(glove_model.vector_size))

In [ ]:
%%time
def model_train(model, train_data, train_labels):
    pipe_globe = Pipeline([
        ('M', model)
        ])
    pipe_globe.fit(train_data, train_labels)
    return pipe_globe

trained_ml_models = []
for i in ml_models:
    tt = model_train(i,globe_text_embeddings, train_labels)
    trained_ml_models.append(tt)

# Test dataset

In [ ]:
test_texts = raw_test_data.clean_text.tolist()
test_labels = raw_test_data.labels.tolist()

In [ ]:
%%time
globe_text_embeddings = []

for text in test_texts:
    tokens = text.split()
    token_embeddings = []
    for token in tokens:
        if token in glove_model:
            token_embedding = glove_model[token]
            token_embeddings.append(token_embedding)
    if token_embeddings:
        text_embedding = np.mean(token_embeddings, axis=0)
        globe_text_embeddings.append(text_embedding)
    else:
        globe_text_embeddings.append(np.zeros(glove_model.vector_size))

In [ ]:
##test data
pr_list = []
re_list = []
f1_list = []
accuracy_list = []


def print_results(md, x_test, y_test, name):
#     print("\n\nClassifier: ", name)
    pred_y = md.predict(x_test)
    true, pred = y_test, pred_y
#     print(confusion_matrix(true,pred))
#     print(classification_report(true,pred))
#     print("\nPrecison : ",precision_score(pred,true, average = 'weighted'))
#     print("\nRecall : ",recall_score(pred,true,  average = 'weighted'))
#     print("\nF1 : ",f1_score(pred,true,  average = 'weighted'))

    pr_list.append((round(precision_score(y_test, pred_y, average='weighted'),4)*100)) 
    re_list.append((round(recall_score(y_test, pred_y, average='weighted'),4)*100))
    f1_list.append((round(f1_score(y_test, pred_y, average='weighted'),4)*100))
    accuracy_list.append(accuracy_score(y_test, pred_y, normalize=True)*100)

for i in range(len(trained_ml_models)):
    md = trained_ml_models[i]
    name = model_names[i]
    print_results(md, globe_text_embeddings, test_labels, name)

performance_matrix = pd.DataFrame({'Precision':pr_list,
                                   'Recall':re_list,'F1 Score':f1_list, 'Accuracy':accuracy_list},
                                  index =model_names)
performance_matrix

# validation dataset

In [ ]:
val_texts = raw_val_data.clean_text.tolist()
val_labels = raw_val_data.labels.tolist()

In [ ]:
%%time
globe_text_embeddings = []

for text in val_texts:
    tokens = text.split()
    token_embeddings = []
    for token in tokens:
        if token in glove_model:
            token_embedding = glove_model[token]
            token_embeddings.append(token_embedding)
    if token_embeddings:
        text_embedding = np.mean(token_embeddings, axis=0)
        globe_text_embeddings.append(text_embedding)
    else:
        globe_text_embeddings.append(np.zeros(glove_model.vector_size))

In [ ]:
##val data
pr_list = []
re_list = []
f1_list = []
accuracy_list = []


def print_results(md, x_test, y_test, name):
#     print("\n\nClassifier: ", name)
    pred_y = md.predict(x_test)
    true, pred = y_test, pred_y
#     print(confusion_matrix(true,pred))
#     print(classification_report(true,pred))
#     print("\nPrecison : ",precision_score(pred,true, average = 'weighted'))
#     print("\nRecall : ",recall_score(pred,true,  average = 'weighted'))
#     print("\nF1 : ",f1_score(pred,true,  average = 'weighted'))

    pr_list.append((round(precision_score(y_test, pred_y, average='weighted'),4)*100)) 
    re_list.append((round(recall_score(y_test, pred_y, average='weighted'),4)*100))
    f1_list.append((round(f1_score(y_test, pred_y, average='weighted'),4)*100))
    accuracy_list.append(accuracy_score(y_test, pred_y, normalize=True)*100)

for i in range(len(trained_ml_models)):
    md = trained_ml_models[i]
    name = model_names[i]
    print_results(md, globe_text_embeddings,val_labels, name)

performance_matrix = pd.DataFrame({'Precision':pr_list,
                                   'Recall':re_list,'F1 Score':f1_list, 'Accuracy':accuracy_list},
                                  index =model_names)
performance_matrix

# CNN with TFIDF Vectorizer

In [195]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Embedding
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from keras import models
from keras import layers
import tensorflow as tf
from keras.applications import VGG16,VGG19,inception_v3
from keras import backend as K
from keras.utils import plot_model

from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization,MaxPooling2D,BatchNormalization,\
                        Permute, TimeDistributed, Bidirectional,GRU, SimpleRNN,\
LSTM, GlobalAveragePooling2D, SeparableConv2D, ZeroPadding2D, Convolution2D, ZeroPadding2D,Reshape,\
Conv2DTranspose, LeakyReLU, Conv1D, AveragePooling1D, MaxPooling1D
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import keras

In [196]:
output_labels = 6
compile_metrics = ["accuracy"]
compile_loss = "categorical_crossentropy"
compile_optimizer = "adam"

In [197]:
%%time

# Reshape the TF-IDF vectors for input to the CNN
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,3), use_idf = True, tokenizer=lambda x: x.split(), max_features=20000) # Adjust ngram_range and features as needed
train_tfidf = tfidf_vectorizer.fit_transform(train_texts)

Scaler_Function = StandardScaler(with_mean=False)
train_tfidf = Scaler_Function.fit_transform(train_tfidf)

train_tfidf = train_tfidf.toarray()
train_tfidf = train_tfidf.reshape(train_tfidf.shape[0], train_tfidf.shape[1], 1)

# train_tfidf = train_tfidf.values
print(train_tfidf.shape)

C:\Users\Taufiq\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


(4994, 20000, 1)
CPU times: total: 891 ms
Wall time: 900 ms


In [198]:
# label_encoder = LabelEncoder()
OHE_Function = OneHotEncoder()

emotion_labels_encoded = OHE_Function.fit_transform(np.array(train_labels).reshape(-1,1)).toarray()

emotion_labels_encoded.shape

(4994, 6)

In [199]:
# Scaler_Function = StandardScaler(with_mean=False)

# train_tfidf = Scaler_Function.fit_transform(train_tfidf)

# train_tfidf.shape

In [200]:
# train_tfidf = train_tfidf.reshape((-1, 1))
# train_tfidf = np.expand_dims(train_tfidf,axis=0)

# print(train_tfidf.shape)

In [201]:
Call_Back_Early_Stop = tf.keras.callbacks.EarlyStopping(monitor="loss",patience=3,mode="min")
Call_Back_Check = tf.keras.callbacks.ModelCheckpoint(monitor="val_accuracy",
                                                     save_best_only=True,
                                                     filepath="./my_model")

In [202]:
Model_Conv1D = Sequential()
#
Model_Conv1D.add(Conv1D(512,5,strides=1,padding="same",activation="relu",input_shape=(train_tfidf.shape[1],1)))
Model_Conv1D.add(BatchNormalization())
Model_Conv1D.add(MaxPooling1D(3,strides=2,padding="same"))
#
Model_Conv1D.add(Conv1D(256,5,strides=1,padding="same",activation="relu"))
Model_Conv1D.add(Dropout(0.3))
Model_Conv1D.add(MaxPooling1D(3,strides=2,padding="same"))


Model_Conv1D.add(Conv1D(128,5,strides=1,padding="same",activation="relu"))
Model_Conv1D.add(Dropout(0.3))
Model_Conv1D.add(MaxPooling1D(3,strides=2,padding="same"))


Model_Conv1D.add(Conv1D(64,5,strides=1,padding="same",activation="relu"))
Model_Conv1D.add(Dropout(0.3))
Model_Conv1D.add(MaxPooling1D(3,strides=2,padding="same"))
#
Model_Conv1D.add(Flatten())
Model_Conv1D.add(Dense(units=1024, activation='relu'))
Model_Conv1D.add(Dropout(0.3))
#
Model_Conv1D.add(Dense(units=output_labels, activation='softmax'))

In [203]:
print(Model_Conv1D.summary())

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_12 (Conv1D)          (None, 20000, 512)        3072      
                                                                 
 batch_normalization_1 (Bat  (None, 20000, 512)        2048      
 chNormalization)                                                
                                                                 
 max_pooling1d_12 (MaxPooli  (None, 10000, 512)        0         
 ng1D)                                                           
                                                                 
 conv1d_13 (Conv1D)          (None, 10000, 256)        655616    
                                                                 
 dropout_4 (Dropout)         (None, 10000, 256)        0         
                                                                 
 max_pooling1d_13 (MaxPooli  (None, 5000, 256)        

In [204]:
%%time

Model_Conv1D.compile(optimizer=compile_optimizer,loss=compile_loss,metrics=compile_metrics)

CPU times: total: 15.6 ms
Wall time: 9.97 ms


# Accuracy find out with test data set:

In [206]:
%%time


# Reshape the TF-IDF vectors for input to the CNN
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,3), use_idf = True, tokenizer=lambda x: x.split(), max_features=20000) # Adjust ngram_range and features as needed
test_tfidf = tfidf_vectorizer.fit_transform(test_texts)

Scaler_Function = StandardScaler(with_mean=False)
test_tfidf = Scaler_Function.fit_transform(test_tfidf)

test_tfidf = test_tfidf.toarray()
test_tfidf = test_tfidf.reshape(test_tfidf.shape[0], test_tfidf.shape[1], 1)

CPU times: total: 109 ms
Wall time: 160 ms


C:\Users\Taufiq\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [207]:
test_tfidf.shape

(625, 20000, 1)

In [208]:
OHE_Function = OneHotEncoder()

emotion_labels_encoded_test = OHE_Function.fit_transform(np.array(test_labels).reshape(-1,1)).toarray()

emotion_labels_encoded_test.shape

(625, 6)

In [ ]:
Conv1D_Model = Model_Conv1D.fit(train_tfidf, emotion_labels_encoded, batch_size=32, epochs=50,
                                validation_data=(test_tfidf, emotion_labels_encoded_test), callbacks=[Call_Back_Early_Stop,Call_Back_Check])

Epoch 1/50


In [127]:
loss, accuracy = model.evaluate(test_tfidf, emotion_labels_encoded_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')


20/20 [==============================] - 4s 181ms/step - loss: 4.3837 - accuracy: 0.2336
Test Accuracy: 23.36%


# Accuracy find out with validation dataset:

In [128]:
%%time


# Reshape the TF-IDF vectors for input to the CNN
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,3), use_idf = True, tokenizer=lambda x: x.split(), max_features=20000) # Adjust ngram_range and features as needed
val_tfidf = tfidf_vectorizer.fit_transform(val_texts)

val_tfidf = val_tfidf.toarray()
val_tfidf = val_tfidf.reshape(val_tfidf.shape[0], val_tfidf.shape[1], 1)

C:\Users\Taufiq\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


CPU times: total: 172 ms
Wall time: 517 ms


In [129]:
label_encoder = LabelEncoder()
emotion_labels_encoded_val = label_encoder.fit_transform(val_labels)

In [130]:
loss, accuracy = model.evaluate(val_tfidf, emotion_labels_encoded_val)
print(f'Test Accuracy: {accuracy * 100:.2f}%')


20/20 [==============================] - 4s 183ms/step - loss: 4.5622 - accuracy: 0.2019
Test Accuracy: 20.19%
